In [ ]:
% load_ext autoreload
% autoreload 2

%matplotlib inline

In [ ]:
from dataset import *
from model import *

In [ ]:
data_test_path = os.path.join("..", "data", "test", "SRAD2018_Test_1")
data_test = get_dataset(data_test_path, is_train=False)

In [ ]:
model = BaselineModel(3, 3, 6, batch_first=True)
model.load_state_dict(torch.load("w.h5"))

In [ ]:
import queue
import threading


def save_seq_func(q):
    def save_seq():
        while True:
            item = q.get()
            if item is None:
                break
            seq_idx, bseq = item
            for i, seq in enumerate(bseq):
                seq_id = data_test.dataset.get_id(seq_idx + i)
                path = os.path.join(result_path, seq_id)
                if not os.path.exists(path):
                    os.makedirs(path)
                for j, img in enumerate(seq):
                    filename = "{}_f{:03d}.png".format(seq_id, j + 1)
                    img.save(os.path.join(path, filename))
            q.task_done()
            print(seq_idx, "ok")
        print("done")
    return save_seq

num_workers = 8
q = queue.Queue(maxsize=128)
threads = []
for i in range(num_workers):
    t = threading.Thread(target=save_seq_func(q))
    t.start()
    threads.append(t)

In [ ]:
import torch.utils.data as data
import torchvision.transforms as transforms

result_path = os.path.join("..", "data", "test", "result")

model = model.cuda()
trans = transforms.Compose([
    BatchApply(BatchApply(transforms.ToPILImage())),
    BatchApply(BatchApply(transforms.Resize(501))),
])

loader_test = data.DataLoader(data_test, batch_size=4)
i = 0
n = 0
print_cnt = 100
for x in loader_test:
    x = x.cuda()
    
    x = model(x)
    x = x.cpu()
    
    x = trans(x)
    
    q.put((i, x))
    print(i)
    i += len(x)
    n += len(x)
    if n > 100:
        n -= 100
        print(i / len(data_test))

In [ ]:
q.join()

for i in range(num_workers):
    q.put(None)
for t in threads:
    t.join()